In [ ]:
import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib
import sys
#scieżka do chrome driver (selenium)
DRIVER_PATH = sys.path[0] + r"\chromedriver.exe"


def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    '''
    Funkcja zwraca adresy url obrazow
    '''
    def scroll_to_end(wd):
        '''
        Funkcja scrolluje wyniki wyszukiwania w dół
        '''
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # link do wyszukiwarki obrazow google
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))
    #deklaracja pustej sekwencji i zmiennych do liczenia
    image_urls = set()
    image_count = 0
    results_start = 0
    #tak dlugo jak liczba obrazow nie przekroczy maksymalnej liczby obrazow jakich chcemy wyszukac...
    while image_count < max_links_to_fetch:
        #scrolluj 
        scroll_to_end(wd)

        # znaleznienie miniaturek zdjec w wynikach wyszukiwania google
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # klikniecie w miniaturke zeby otworzyc obraz wyszukiwaniu
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # pobieranie linkow do obrazow znalezionych w google    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                # jezeli w atrybucie src mamy protokol http dodajemyadres url do sekwencji z adresami url
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break

    return image_urls

def persist_image(folder_path:str,file_name:str,url:str):
    try:
        #przejscie do url dla obrazu
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        #Zapis pobranych obrazow w folderze
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["Robert Biedroń",
               "janusz korwin-mikke",
               "Andrzej Duda",
               "rafał trzaskowski",
               "szymon hołownia", 
               "Beata Szydło",
               "Bronisław Komorowski",
               "Anna Grodzka",
               "Leszek Miller",
               "Ryszard Kalisz"]
    for query in queries:
        #otwieranie google
        wd.get('https://google.com')
        #znalezienie na stronie search boxu
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        #wpisanie elementu do search boxu
        search_box.send_keys(query)
        #wywolanie funckji fetch_image_url
        links = fetch_image_urls(query,100,wd)
        images_path = sys.path[0]
        for i in links:
            persist_image(images_path,query,i)
    wd.quit()